In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import os
import re
import shutil

In [3]:
#Labels for train and test data for both calc and mass cases

## Train data labels
calc_all_train = pd.read_csv("calc_case_description_train_set.csv")
calc_train = calc_all_train
calc_train['image_name'] = calc_all_train.patient_id + '_' + calc_all_train['left or right breast'] + '_' + calc_all_train['image view'] + '.png'
calc_train.drop(["image file path","cropped image file path","ROI mask file path"], axis=1, inplace=True)
calc_train.columns = ["Patient_ID","Breast_Density","Side_L_R","Image View","Abnormality_ID","Abnormality_Type","Mass_Shape","Mass_Margins","Assessment","Pathology", "Subtlety","Image_Name"]


In [4]:
mass_train = pd.read_csv("mass_case_description_train_set.csv")
mass_all_train = mass_train
mass_train['image_name'] = mass_all_train.patient_id + '_' + mass_all_train['left or right breast'] + '_' + mass_all_train['image view'] + '.png'
mass_train.drop(["image file path","cropped image file path","ROI mask file path"], axis=1, inplace=True)
mass_train.columns = ["Patient_ID","Breast_Density","Side_L_R","Image View","Abnormality_ID","Abnormality_Type","Mass_Shape","Mass_Margins","Assessment","Pathology", "Subtlety","Image_Name"]


In [5]:
## Test data labels
calc_test = pd.read_csv("calc_case_description_test_set.csv")
calc_all_test = calc_test
calc_test['image_name'] = calc_all_test.patient_id + '_' + calc_all_test['left or right breast'] + '_' + calc_all_test['image view'] + '.png'
calc_test.drop(["image file path","cropped image file path","ROI mask file path"], axis=1, inplace=True)
calc_test.columns = ["Patient_ID","Breast_Density","Side_L_R","Image View","Abnormality_ID","Abnormality_Type","Mass_Shape","Mass_Margins","Assessment","Pathology", "Subtlety","Image_Name"]

mass_test = pd.read_csv("mass_case_description_test_set.csv")
mass_all_test = mass_test
mass_test['image_name'] = mass_all_test.patient_id + '_' + mass_all_test['left or right breast'] + '_' + mass_all_test['image view'] + '.png'
mass_test.drop(["image file path","cropped image file path","ROI mask file path"], axis=1, inplace=True)
mass_test.columns = ["Patient_ID","Breast_Density","Side_L_R","Image View","Abnormality_ID","Abnormality_Type","Mass_Shape","Mass_Margins","Assessment","Pathology", "Subtlety","Image_Name"]

In [6]:

# Train and Test label
train_label = pd.concat([calc_all_train, mass_all_train], axis = 0)
train_label['Pathology'][train_label['Pathology'] == 'BENIGN_WITHOUT_CALLBACK'] = 'BENIGN'
train_label['Class'] = train_label['Pathology'] + '_' + train_label['Abnormality_Type']

test_label = pd.concat([calc_all_test, mass_all_test], axis = 0)
test_label['Pathology'][test_label['Pathology'] == 'BENIGN_WITHOUT_CALLBACK'] = 'BENIGN'
test_label['Class'] = test_label['Pathology'] + '_' + test_label['Abnormality_Type']


C:\Users\dahyun\anaconda3\envs\mammo\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\dahyun\anaconda3\envs\mammo\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [7]:

# Set image_name to be the index
train_label.set_index("Image_Name", inplace = True)
test_label.set_index("Image_Name", inplace = True)

#test_label.head()

In [8]:
train_label.loc['P_00034_RIGHT_CC.png']['Class']

Image_Name
P_00034_RIGHT_CC.png    MALIGNANT_calcification
P_00034_RIGHT_CC.png             MALIGNANT_mass
Name: Class, dtype: object

In [9]:
test_label.loc['P_00202_RIGHT_CC.png']['Class']

Image_Name
P_00202_RIGHT_CC.png    MALIGNANT_calcification
P_00202_RIGHT_CC.png             MALIGNANT_mass
Name: Class, dtype: object

In [12]:
#train data
CBIS_train_patches = np.load(os.path.join("npy_new6", "train_mass_patch.npy" ))
CBIS_train_labels = np.load(os.path.join("npy_new6", "train_mass_Lbl.npy" ))
CBIS_train_FNs = np.load(os.path.join("npy_new6", "train_mass_FN.npy" ))

#test data
CBIS_test_patches = np.load(os.path.join("npy_new6", "test_mass_patch.npy" ))
CBIS_test_labels = np.load(os.path.join("npy_new6", "test_mass_Lbl.npy" ))
CBIS_test_FNs = np.load(os.path.join("npy_new6", "test_mass_FN.npy" ))



print("Abnaormal train Patches:", CBIS_train_patches.shape)
print("Abnaormal train Labels:", CBIS_train_labels.shape)
print("Abnaormal train File Names:", CBIS_train_FNs.shape)
print("\n")
print("Abnaormal test Patches:", CBIS_test_patches.shape)
print("Abnaormal test Labels:", CBIS_test_labels.shape)
print("Abnaormal test File Names:", CBIS_test_FNs.shape)

Abnaormal train Patches: (1546, 256, 256, 1)
Abnaormal train Labels: (1546,)
Abnaormal train File Names: (1546,)


Abnaormal test Patches: (326, 256, 256, 1)
Abnaormal test Labels: (326,)
Abnaormal test File Names: (326,)


In [13]:
#combine train and test data 

CBIS_all_patches = np.concatenate([CBIS_train_patches, CBIS_test_patches], axis=0)
CBIS_all_labels = np.concatenate([CBIS_train_labels, CBIS_test_labels], axis=0)
CBIS_all_FNs = np.concatenate([CBIS_train_FNs, CBIS_test_FNs], axis=0)

CBIS_all_patches, CBIS_all_labels, CBIS_all_FNs = \
shuffle(CBIS_all_patches, CBIS_all_labels, CBIS_all_FNs, random_state=19510705)

In [14]:
CBIS_all_labels[1685]

'BENIGN_calcification'

In [15]:
CBIS_all_labels[1685][-13:]=='calcification'

True

In [15]:
diff = []
for i in range(0,CBIS_all_labels.size,1):
    print(str(i) + ' ' + CBIS_all_labels[i])
    if(CBIS_all_labels[i][-13:]=='calcification'):
        diff.append(i)

0 BENIGN_calcification
1 MALIGNANT_calcification
2 BENIGN_calcification
3 MALIGNANT_calcification
4 MALIGNANT_calcification
5 BENIGN_calcification
6 MALIGNANT_calcification
7 BENIGN_calcification
8 BENIGN_calcification
9 BENIGN_calcification
10 MALIGNANT_calcification
11 BENIGN_calcification
12 MALIGNANT_calcification
13 BENIGN_calcification
14 BENIGN_calcification
15 MALIGNANT_calcification
16 BENIGN_calcification
17 MALIGNANT_calcification
18 BENIGN_calcification
19 MALIGNANT_calcification
20 BENIGN_calcification
21 BENIGN_calcification
22 BENIGN_calcification
23 BENIGN_calcification
24 MALIGNANT_calcification
25 BENIGN_calcification
26 MALIGNANT_calcification
27 BENIGN_calcification
28 BENIGN_calcification
29 BENIGN_calcification
30 BENIGN_calcification
31 BENIGN_calcification
32 BENIGN_calcification
33 BENIGN_calcification
34 BENIGN_calcification
35 MALIGNANT_calcification
36 BENIGN_calcification
37 BENIGN_calcification
38 BENIGN_calcification
39 BENIGN_calcification
40 BENIGN_calc

In [16]:
diff

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [18]:
for i in diff :
    if(CBIS_all_FNs[i][5:9] == 'Test'):
        fn = CBIS_all_FNs[i][10:]
        new = test_label.loc[fn]['Class'][-1]
    else :
        fn = CBIS_all_FNs[i][14:]
        new = train_label.loc[fn]['Class'][-1]
    CBIS_all_labels[i] = new

CBIS_all_labels[1685] = 'MALIGNANT_mass'

index = 1685
print(CBIS_all_labels[index])
print(CBIS_all_FNs[index])

In [17]:
#split the combined data into train and test
train_patches, test_patches, train_labels, test_labels, train_FNs, test_FNs = \
train_test_split(CBIS_all_patches, CBIS_all_labels, CBIS_all_FNs, test_size = 0.183565, random_state=19430727)

In [18]:
#train data
train_images = train_patches
train_labels = train_labels
train_FNs = train_FNs

#test data
test_images = test_patches
test_labels = test_labels
test_FNs = test_FNs

In [19]:
test_images.shape

(344, 256, 256, 1)

In [20]:
le = preprocessing.LabelEncoder()
le.fit(train_labels)

LabelEncoder()

In [21]:
list(le.classes_)

['BENIGN_calcification', 'MALIGNANT_calcification']

In [22]:
#Convert Normal to 0 
train_labels_en = le.transform(train_labels)
#train_labels_en[train_labels_en==]=0

test_labels_en = le.transform(test_labels)
#test_labels_en[test_labels_en==5]=0

In [23]:
classes = le.classes_

In [24]:
train_bin_labels = np.zeros(len(train_labels_en)).astype(np.int32)
train_bin_labels[train_labels_en != 0] = 1

test_bin_labels = np.zeros(len(test_labels_en)).astype(np.int32)
test_bin_labels[test_labels_en != 0] = 1

In [25]:
train_labels_en

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [26]:
np.save(os.path.join("Label_mass", "train_labels_en.npy"), train_labels_en)
np.save(os.path.join("Label_mass", "test_labels_en.npy"), test_labels_en)
np.save(os.path.join("Label_mass", "train_bin_labels.npy"), train_bin_labels)
np.save(os.path.join("Label_mass", "test_bin_labels.npy"), test_bin_labels)

In [27]:
#X_val, X_test, y_val, y_test, y_val_multi, y_test_multi = \
#    train_test_split(test_images, test_bin_labels, test_labels_en, test_size=0.5, random_state=19730104)
X_train, y_train, y_train_multi = \
     shuffle(train_images, train_bin_labels, train_labels_en, random_state=100)
X_test, y_test, y_test_multi = \
     shuffle(test_images, test_bin_labels, test_labels_en, random_state=100)

In [28]:
y_test_multi

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,

In [29]:
len(X_train)

1528

In [31]:
np.save(os.path.join("Data_calc", 'X_train.npy'), X_train)
np.save(os.path.join("Data_calc", 'y_train.npy'), y_train)
np.save(os.path.join("Data_calc", 'train_labels_multi.npy'), y_train_multi)

np.save(os.path.join("Data_calc", 'X_test.npy'), X_test)
np.save(os.path.join("Data_calc", 'y_test.npy'), y_test)
np.save(os.path.join("Data_calc", 'y_test_labels_multi.npy'), y_test_multi)

In [ ]:
 images = np.load(os.path.join('Data_mass', 'X_train.npy'))
    labels = np.load(os.path.join('Data_mass', 'train_labels_multi.npy'))

In [42]:
#train data
CBIS_calc_train_patches = np.load(os.path.join('Data_calc', 'X_train.npy'))
CBIS_calc_train_labels = np.load(os.path.join('Data_calc', 'y_train.npy'))
CBIS_calc_train_FNs = np.load(os.path.join('Data_calc', 'train_labels_multi.npy'))

#test data
CBIS_calc_test_patches = np.load(os.path.join('Data_calc', 'X_test.npy'))
CBIS_calc_test_labels = np.load(os.path.join('Data_calc', 'y_test.npy'))
CBIS_calc_test_FNs = np.load(os.path.join('Data_calc', 'y_test_labels_multi.npy'))

#train data
CBIS_mass_train_patches = np.load(os.path.join('Data_mass', 'X_train.npy'))
CBIS_mass_train_labels = np.load(os.path.join('Data_mass', 'y_train.npy'))
CBIS_mass_train_FNs = np.load(os.path.join('Data_mass', 'train_labels_multi.npy'))

#test data
CBIS_mass_test_patches = np.load(os.path.join('Data_mass', 'X_test.npy'))
CBIS_mass_test_labels = np.load(os.path.join('Data_mass', 'y_test.npy'))
CBIS_mass_test_FNs = np.load(os.path.join('Data_mass', 'y_test_labels_multi.npy'))

In [43]:
#데이터 합치기
train_patch = np.concatenate([CBIS_calc_train_patches, CBIS_mass_train_patches], axis=0)
train_Lbl = np.concatenate([CBIS_calc_train_labels, CBIS_mass_train_labels], axis=0)
train_FN = np.concatenate([CBIS_calc_train_FNs, CBIS_mass_train_FNs], axis=0)

test_patch = np.concatenate([CBIS_calc_test_patches, CBIS_mass_test_patches], axis=0)
test_Lbl = np.concatenate([CBIS_calc_test_labels, CBIS_mass_test_labels], axis=0)
test_FN = np.concatenate([CBIS_calc_test_FNs, CBIS_mass_test_FNs], axis=0)

In [45]:
train_patch.shape

(2912, 256, 256, 1)

In [46]:
np.save(os.path.join("Data_new7", 'X_train.npy'), train_patch)
np.save(os.path.join("Data_new7", 'y_train.npy'), train_Lbl)
np.save(os.path.join("Data_new7", 'train_labels_multi.npy'), train_FN)

np.save(os.path.join("Data_new7", 'X_test.npy'), test_patch)
np.save(os.path.join("Data_new7", 'y_test.npy'), test_Lbl)
np.save(os.path.join("Data_new7", 'y_test_labels_multi.npy'), test_FN)